In [4]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 12 14:04:52 2019

@author: julien.linder
"""

import cv2

if __name__=="__main__":
    img = cv2.imread("C:/DEV/traitement d'images/serie4e/Images/TroisBallesRougeBleueBlanche.png")
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    
    mask = cv2.inRange(hsv, (110, 50, 50), (130, 255, 255))
    res = cv2.bitwise_and(img, img, mask=mask)
    
    cv2.imshow('img', img)
    cv2.imshow('mask', mask)
    cv2.imshow('res', res)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (18, 17))
    
    erode = cv2.erode(mask, kernel, iterations=2)
    cv2.imshow('erode', erode)
    
    dilate = cv2.dilate(erode, kernel, iterations=2)
    cv2.imshow('dilate', dilate)
    res2 = cv2.bitwise_and(img, img, mask=dilate)
    cv2.imshow('res2', res2)
    
    cv2.waitKey(0)

In [10]:
import cv2

In [57]:
src = "unebouteille"
img = cv2.imread("C:/DEV/traitement d'images/findbottle/" + src + ".jpg")
#print(img)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#print(hsv)
mask = cv2.inRange(hsv, (0, 0, 0), (100, 200, 100))

In [58]:
cv2.imshow("image", img)

cv2.imshow('hsv', hsv)

cv2.imshow('mask', mask)

cv2.waitKey(0)

-1